 <font size="6">Сегментация и детектирование</font>

# Задачи компьютерного зрения

В ходе предыдущих занятий, мы подробно рассмотрели задачу классификации изображений.


Но порой недостаточно знать, что на изображении есть объект определенного класса. Важно, где именно расположен объект. В ряде случаев нужно знать еще и точные границы объекта. Например, если речь идет о рентгеновском снимке или изображении клеток ткани, полученном с микроскопа.


<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/classification_semantic_segmentation.png" width="600">


Определение того, какие фрагменты изображения принадлежат объектам определенных классов - это задача **сегментации** (segmentation).


Если нас интересуют не индивидуальные объекты, а только тип(класс) объекта которым занят конкретный пиксел (как в случае с клетками под микроскопом) то говорят о **семантической сегментации** (semantic segmentation)

Если нас интересуют конкретные объекты, и при этом достаточно знать только область, в которой объект локализован, то это задача **детектирования** (Detection)

В качестве примера такой задачи можно рассмотреть подсчет количество китов на спутниковом снимке.


Если же важны и индивидуальные объекты и их точные границы, то это уже задача **Instance segmentation**

## Dataset COCO — Common Objects in Context

Прежде чем говорить о способах решения этих задач, надо разобраться с форматами входных данных. Сделаем это на примере COCO.

Один из наиболее популярных датасатов, содержащий данные для сегментации и детектирования. Он содержит более трёхсот тысяч изображений, большая часть из которых размечена и содержит следующую информацию:
- Категории
- Маски
- Ограничивающие боксы (*bounding boxes*)
- Описания (*captions*)
- Ключевые точки (*keypoints*)
- И многое другое

Формат разметки изображений, использованный в этом датасете, нередко используется и в других наборах данных. Как правило, он упоминается просто как "COCO format".

Загрузим датасет.

In [ ]:
import requests, zipfile, io

r = requests.get(
    "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

Для работы с датасетом используется пакет `pycocotools`.

[Подробнее о том, как создать свой COCO датасет с нуля](https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch).

In [ ]:
from pycocotools.coco import COCO

coco = COCO("annotations/instances_val2017.json")

Рассмотрим формат аннотаций на примере одной записи.

In [ ]:
catIds = coco.getCatIds(catNms=["cat"])  # cats IDs
print('class ID(cat) = %i' % catIds[0])

imgIds = coco.getImgIds(catIds=catIds)  # Filtering dataset by tag
print("All images: %i" % len(imgIds))

Рассмотрим метаданные.

In [ ]:
img_list = coco.loadImgs(imgIds[0])  # 1 example
img = img_list[0]
img

Посмотрим на изображение.

In [ ]:
import skimage.io as io
import matplotlib.pyplot as plt

I = io.imread(img["coco_url"])
plt.axis("off")
plt.imshow(I)
plt.show()

Сконвертируем в PIL формат для удобства дальнейшей работы.

In [ ]:
from PIL import Image
from io import BytesIO

def coco2pil(url):
    print(url)
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

pil_img = coco2pil(img["coco_url"])
plt.axis("off")
plt.imshow(pil_img)
plt.show()

### Категории в COCO

Давайте посмотрим на примеры категорий в нашем датасете. Отобразим каждую 10ую категорию.

In [ ]:
cats = coco.loadCats(coco.getCatIds())  # loading categories
num2cat = {}  
print("COCO categories: ")
for cat in cats:
    num2cat[cat["id"]] = cat["name"]
    if cat["id"] in range(0, 80, 10):
        print(cat["id"], ":", cat["name"], end="   \n")

В датасете также есть категория **0**. Ее используют для обозначения класса фона.

Также существуют надкатегории. 

In [ ]:
print(f'cats[2]: {cats[2]}')
print(f'cats[3]: {cats[3]}')

nms = set([cat["supercategory"] for cat in cats])
print("COCO supercategories: \n{}".format("\n".join(nms)))

### Разметка данных

Помимо метаданных нам доступна разметка ([подробнее о разметке](https://cocodataset.org/#format-data)), давайте её загрузим и отобразим.

In [ ]:
annIds = coco.getAnnIds(imgIds=img["id"])
anns = coco.loadAnns(annIds)

plt.imshow(I)
plt.axis("off")
coco.showAnns(anns)
plt.show()

На изображении можно увидеть разметку пикселей изображения по классам. То есть, пиксели из объектов, относящихся к интересующим классам, приписываются к классу этого объекта. К примеру, можно увидеть объекты двух классов: "cat" и "keyboard". 

Давайте теперь посмотрим, из чего состоит разметка.

In [ ]:
def dump_anns(anns):
    for i, a in enumerate(anns):
        print(f"\n#{i}")
        for k in a.keys():
            if k == "category_id" and num2cat.get(a[k], None):
                print(k, ": ", a[k], num2cat[a[k]])  # Show cat. name
            else:
                print(k, ": ", a[k])

dump_anns(anns)

Заметим, что аннотация изображения может состоять из описаний нескольких объектов, каждое из которых содержит следующую информацию:
* `segmentation` - последовательность пар чисел ($x$, $y$), координат каждой из вершин "оболочки" объекта;
* `area` - площадь объекта;
* `iscrowd` - информация о том, находится в оболочке один объект или же несколько, но слишком много для пообъектной разметки (толпа людей, к примеру);
* `image_id` - идентификатор изображения, к которому принадлежит описываемый объект;
* `bbox` - *будет рассмотрен далее в ходе лекции*;
* `category_id` - идентификатор категории, к которой относится данный объект;
* `id` - идентификатор самого объекта.

Попробуем посмотреть на пример, в котором `iscrowd = True` .

In [ ]:
plt.rcParams["figure.figsize"] = (120, 60)

catIds = coco.getCatIds(catNms=["people"])
annIds = coco.getAnnIds(catIds=catIds, iscrowd=True)
anns = coco.loadAnns(annIds[0:1])

dump_anns(anns)
img = coco.loadImgs(anns[0]["image_id"])[0]
I = io.imread(img["coco_url"])
plt.figure(figsize=(10, 10))
plt.imshow(I)
coco.showAnns(anns)  # People in the stands
seg = anns[0]["segmentation"]
print("Counts", len(seg["counts"]))
print("Size", seg["size"])
plt.axis("off")
plt.show()

Используя методы из `pycocotools`, можно с лёгкостью преобразовать набор вершин "оболочки" сегментируемого объекта в более удобный для отображения вид - в маску объекта.

In [ ]:
import numpy as np

annIds = coco.getAnnIds(imgIds=[448263])
anns = coco.loadAnns(annIds)
msk = np.zeros(seg["size"])

fig, ax = plt.subplots(nrows=4, ncols=4, figsize=(10, 10))

i = 0
for row in range(4):
    for col in range(4):
        msk = coco.annToMask(anns[i])
        ax[row, col].imshow(msk, cmap = 'gray')
        ax[row, col].set_title(num2cat[anns[i]["category_id"]])
        ax[row, col].axis("off")
        i += 1
        
plt.show()

В некоторых случаях, попиксельная разметка изображения может быть избыточной - к примеру, в случае, если необходимо посчитать количество человек на изображении, достаточно просто каким-то образом промаркировать каждого из них, после чего посчитать количество наших "отметок". Одним из вариантов маркировки является "обведение" объекта рамкой (`bounding boxes`), внутри которой он находится. Такая информация об объектах также сохранена в аннотациях формата COCO.

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

annIds = coco.getAnnIds(imgIds=[448263])
anns = coco.loadAnns(annIds)

RGB_img = cv2.cvtColor(I, cv2.COLOR_BGR2RGB)

for i in range(len(anns)):
    x, y, width, heigth = anns[i]["bbox"]
    x, y, width, heigth = int(x), int(y), int(width), int(heigth)
    if anns[i]["category_id"] == 1: # person
        color = (255, 255, 255)
    if anns[i]["category_id"] == 40: # glove
        color = (0, 255, 0)
    RGB_img = cv2.rectangle(RGB_img, (x, y), (x + width, y + heigth), color, 2)
cv2_imshow(RGB_img)


Еще более глубокое понимание разметки:

Что такое [run-length encoding - RLE](https://en.wikipedia.org/wiki/Run-length_encoding)

[Видео-разбор](https://www.youtube.com/watch?v=h6s61a_pqfM)

# Семантическая сегментация (*Semantic segmentation*)

*Сегментация изображения — задача поиска групп пикселей, каждая из которых характеризует один смысловой объект.*


Технически это выглядит так.

Есть набор изображений:

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/semantic_segmentation_1.png" width="300">

Для каждого изображения есть маска W x H:

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/semantic_segmentation_2.png" width="300">

Маска задает класс объекта для каждого пикселя:
[ x,y - > class_num ]


Набор таких изображений с масками это и есть наш датасет, на нем мы учимся.


На вход модель получает новое изображение:

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/semantic_segmentation_3.png" width="600">

и должна предсказать метку класса для каждого пикселя (маску).

[ x,y - > class_num ] 





## Способы предсказания класса для каждого пикселя

Давайте подумаем о том, как такую задачу можно решить.

Из самой постановки задачи видно, что это задача классификации. Только не 
всего изображения, а каждого пикселя.

### a) Наивный




Простейшим вариантом решения является использование так называемого "скользящего окна" - последовательное рассмотрение фрагментов изображения. В данном случае, интересующими фрагментами будут небольшие зоны, окружающие каждый из пикселей изображения. К каждому из таких фрагментов применяется свёрточная нейронная сеть, предсказывающая, к какому классу относится центральный пиксель.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/naive_way_predict_pixel_class.png" width="600">

### б) Разумный

Понятно, что запускать классификатор для каждого пикселя абсолютно неэффективно, так как для одного изображения потребуется $H*W$ запусков.

Можно пойти другим путем: получить карту признаков и по ней делать предсказание для всех пикселей разом.




Для этого потребуется поменять привычную нам архитектуру сверточной сети следующим образом:

* убрать слои уменьшающие пространственные размеры

* убрать линейный слой в конце, заменив его сверточным





<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/reasonable_way_predict_pixel_class.png" width="700">

Теперь пространственные размеры выхода (W,H) будут равны ширине и высоте исходного изображения.

 
Количество выходных каналов будет равно количеству классов, которые мы учимся предсказывать. 


Тогда можно использовать значения каждой из карт активаций, на выходе последнего слоя сети, как ненормированное значение вероятности принадлежности(core) каждого из пикселей к тому или иному классу. 

То есть номер канала с наибольшим значением будет соответствовать классу объекта, который изображает данный пиксел.

In [ ]:
import torch
 
last_layer_output = torch.randn((3, 32, 32)) # class_num, W,H
print(last_layer_output.shape, last_layer_output[:, :3, :3]) # activation slice
mask = torch.argmax(last_layer_output, dim=0) # class_nums prediction
print(mask.shape)
print(mask[:5, :5])

Чтобы на выходе сети получить нужное количество каналов, используется свертка 1x1.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/1x1_kernel_size_fully_connected_layer.png" width="500">

В лекции про сверточные сети мы говорили о том, что свертку 1x1 можно рассматривать как аналог полносвязанного слоя. Именно так она тут и работает.

**Проблемы:**
- так как нужно сохранить большое рецептивное поле, требуется много сверточных слоев ($L$ раз свёртка $3\times3$ $\to$ рецептивное поле $(1+2L)\times(1+2L)$);
- свертки медленно работают на полноразмерных картах активации.

### в) Эффективный

Сохраним большую часть традиционной структуры сети, включая слои downsampling (снижение пространственных размеров), извлечем основные признаки, а затем восстановим пространственные размеры.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/efficient_way_predict_pixel_class.png" width="800">

Именно такой подход с некоторыми дополнениями используется на практике.

## Автокодировщик

Эта архитектура повторяет архитектуру автокодировщика (autoencoder)

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/encoder_loss.png" width="500">

Такая архитектура довольно популярна и применяется не только для сегментации: 

- сглаживание шума;
- снижение размерности $\to$ вектор-признак;
- генерация данных.

Этому будет посвящена одна из следующих лекций, сейчас же мы детально рассмотрим разжимающий блок.

## Разжимающий слой

Как реализовать декодировщик?

### Интерполяция при увеличении разрешения

Вспомним, как повышают разрешение для обычных изображений, а уже затем перейдем к картам признаков.

Допустим, требуется увеличить изображение размером 2x2 до размера 4x4. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/upsample.png" width="300">


Самый простой способ это сделать:
- создать пустое изображение нужного размера
- поделить его на 4 части (по числу пикселей исходного изображения)
- каждую часть закрасить цветом соответствующего пикселя маленького изображения

Это интерполяция методом ближайшего соседа(Nearest neighbor interpolation).

Естественно она реализована в пакете [Pillow Image](https://pillow.readthedocs.io/en/stable/reference/Image.html):







In [ ]:
!wget -q https://edunet.kea.su/repo/EduNet-content/L12/out/semantic_segmentation_1.png -O cat.jpg

In [ ]:
from PIL import Image

img = Image.open("cat.jpg")
img

In [ ]:
import PIL
import numpy as np

new_size = np.array(img.size) * 2
resized = img.resize(new_size, resample=PIL.Image.NEAREST)
resized

Это не самый лучший вид интерполяции: на изображении появились артефакты (квадраты).

Чтобы избавиться от таких артефактов можно, значениям новых пикселей присваивать интерполированное значение яркости исходных.



<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/change_size_of_image.png" width="800">

Если для интерполяции используются значения четырех соседних пикселей это биленейная интерполяция. В качестве интерполированного значения используется взвешенное среднее этих четырёх пикселей. 


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def img_to_heatmap(img, ax, title): #Magik method to show img as heatmap
  ax.axis('off')
  ax.set_title(title)
  array = np.array(img)
  array = array[None, None, :]
  sns.heatmap(array[0][0], annot=True, ax=ax, lw=1, cbar=False)


# Fake image
raw = np.array([[1, 3, 0, 1], [3, 3, 3, 7], [8, 1, 8, 7], [6, 1, 1, 1]], dtype=np.uint8)
pil = Image.fromarray(raw)

interp_nn = pil.resize((8, 8), resample=PIL.Image.NEAREST) 
interp_bl = pil.resize((8, 8), resample=PIL.Image.BILINEAR)

# plot result
fig, ax = plt.subplots(ncols=3, figsize=(15, 5), sharex=True, sharey=True)
img_to_heatmap(raw,ax[0], 'Raster dataset')
img_to_heatmap(interp_nn,ax[1], 'Nearest neighbor interpolation')
img_to_heatmap(interp_bl,ax[2], 'Bilinear interpolation')
plt.show()

Билинейная Интерполяция позволяет избавиться от резких границ, которые возникают при увеличении методом ближайшего соседа.  Существуют и другие виды интерполяции, использующие большее количество соседних пикселей.





#### Подробнее про интерполяцию

Билинейная интерполяция рассматривает квадрат соседних пикселей. В качестве интерполированного значения используется взвешенное среднее этих четырёх пикселей. 

Отметим, что чем ближе интерполируемое значение находится к одному из известных пикселей, тем больший вес будет у значения этого пикселя при подсчёте взвешенной суммы. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/pixel_iterpolation_2x2.png" width="800">

Иногда окрестность $2\times2$ может быть не достаточна для сохранения важной информации о распределении цветов в окружающих пикселях (пример будет приведён далее). В таких случаях, можно попробовать использовать большую окрестность для интерполяции - к примеру, $4\times4$. Такая интерполяция называется бикубической и имеет более сложную формулу, чем билинейная, однако основной принцип остаётся тем же - чем ближе пиксель к пикселю с известным значением, тем больше "вес" последнего при интерполяции.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/pixel_iterpolation_4x4.png" width="800">

### Upsample в Pytorch

К чему был этот разговор об увеличении картинок?

Оказывается для увеличения пространственного разрешения карт признаков (feature maps) можно применять те же методы что и для изображений.




Для увеличения пространственного разрешения карт признаков(карт активаций), в PyTorch используется класс `Upsample`. В нём доступны все упомянутые методы интерполяции, а также трилинейная интерполяция - аналог билинейной интерполяции, используемый для работы с трёхмерными пространственными данными (к примеру, видео). 

 


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/upsample_pytorch.png" width="1000">

[[doc] nn.Upsample](https://pytorch.org/docs/stable/generated/torch.nn.Upsample.html)

[[doc] nn.functional.interpolate](https://pytorch.org/docs/stable/generated/torch.nn.functional.interpolate.html?highlight=interp#torch.nn.functional.interpolate)


Таким образом, мы можем использовать `Upsample` внутри нашего разжимающего блока.

In [ ]:
from torch import nn
import torchvision.transforms.functional as TF

def upsample(pil, ax, mode="nearest"):
    tensor = TF.to_tensor(pil)
    # Create upsample instance

    if mode == 'nearest':
        upsampler = nn.Upsample(scale_factor=2, mode=mode)
    else:
        upsampler = nn.Upsample(scale_factor=2, mode=mode, align_corners=True)
        
    tensor_128 = upsampler(tensor.unsqueeze(0)) # add batch dimension
    # Convert tensor to pillow
    img_128 = tensor_128.squeeze()
    img_128_pil = TF.to_pil_image(img_128.clamp(min=0, max=1))
    ax.imshow(img_128_pil)
    ax.set_title(mode)
    
# Load and show image in Pillow format
pic = Image.open("cat.jpg") 
pil_64 = pic.resize((64, 64))
fig, ax = plt.subplots(ncols=4, figsize=(15, 5))
ax[0].imshow(pil_64)
ax[0].set_title("Raw")

# Upsample with Pytorch
upsample(pil_64, mode="nearest", ax=ax[1])
upsample(pil_64, mode="bilinear", ax=ax[2])
upsample(pil_64, mode="bicubic", ax=ax[3])
plt.show()

Обратите внимание, что в данном случае каждое из пространственных измерений изображения увеличилось в 2 раза, но при необходимости возможно использовать увеличение в иное в том числе не целое количество раз, используя параметр `scale_factor`

Слои Upsample обычно комбинируют вместе со сверточными, это рекомендованный способ увеличения пространственных размеров карт признаков.

In [ ]:
model = nn.Sequential(
  nn.Upsample(scale_factor=2), 
  nn.Conv2d(3, 16, kernel_size=3, padding=1),
  nn.ReLU()
)

dummy_input = torch.randn((0, 3, 32, 32))
out = model(dummy_input)
print(out.shape)

#### Другие способы "разжать" карту признаков

##### MaxUnpooling

Помимо свёртки, на этапе снижения размерности также используются слои pooling'а. Наиболее популярным вариантом является maxpooling, сохраняющий значение только наибольшего элемента внутри сегмента. Для того чтобы обратить данную операцию субдискретизации, был предложен MaxUnpooling слой.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/maxunpooling.png" width="650">

Данный слой требует сохранения индексов максимальных элементов внутри сегментов - при обратной операции, максимальное значение помещается на место, в котором был максимальный элемент сегмента до соответствующей субдискретизации. Соответственно, каждому слою MaxUnpooling должен соответствовать слой MaxPooling, что визуально можно представить следующим образом:

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/downsample_and_upsample_layers.png" width="650">

[Документация к MaxPool2d](
https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)

[Документация к MaxUnpool2d](
https://pytorch.org/docs/stable/generated/torch.nn.MaxUnpool2d.html?highlight=unpooling)

In [ ]:
import requests
from io import BytesIO

def coco2pil(url):
    print(url)
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

def tensor_show(tensor, title="", ax=ax):
    img = TF.to_pil_image(tensor.squeeze()).convert("RGB")
    ax.set_title(title + str(img.size))
    ax.imshow(img)


pool = nn.MaxPool2d(kernel_size=2, return_indices=True)  # False by default(get indexes to upsample)
unpool = nn.MaxUnpool2d(kernel_size=2)

pil = coco2pil("http://images.cocodataset.org/val2017/000000448263.jpg")

fig, ax = plt.subplots(ncols=5, figsize=(20, 5), sharex=True, sharey=True)

ax[0].set_title("original " + str(pil.size))
ax[0].imshow(pil)
tensor = TF.to_tensor(pil).unsqueeze(0)
print("Orginal shape", tensor.shape)

# Downsample
tensor_half_res, indexes1 = pool(tensor)
tensor_show(tensor_half_res, "1/2 down ", ax=ax[1])

tensor_q_res, indexes2 = pool(tensor_half_res)
tensor_show(tensor_q_res, "1/4 down ", ax=ax[2])
print("Downsample shape", indexes2.shape)

# Upsample
tensor_half_res1 = unpool(tensor_q_res, indexes2)
tensor_show(tensor_half_res1, "1/2 up ", ax=ax[3])


tensor_recovered = unpool(tensor_half_res1, indexes1)
tensor_show(tensor_recovered, "full size up ", ax=ax[4])
print("Upsample shape", tensor_recovered.shape)
plt.show()

##### Transpose convolution

Способы восстановления пространственных размерностей, которые мы рассмотрели, не содержали обучаемых параметров. 


Для повышения пространственного разрешения карты признаков можно использовать операцию Transpose convolution. 

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/simple_convolution.png" width="700"></center>
<center><em>Обычная свертка.</em></center>

Отличие этого метода от свёртки заключается в том, что вместо перемножения каждого входного значения на соответствующий элемент фильтра, все элементы фильтра домножаются на единственное входное значение. 

В результате получается матрица, размеры которой совпадают с размерами фильтра.




<center><img src ="http://edunet.kea.su/repo/EduNet-web_dependencies/L12/transposed_convolutions.png" width="1200"></center>

<center><em>Upsample/transpose convolution.</em></center>

Из них формируется выход слоя. В зонах, где полученные значения "накладываются" друг на друга, они суммируются.

[Блог-пост про 2d свертки с помощью перемножения матриц](https://medium.com/@_init_/an-illustrated-explanation-of-performing-2d-convolutions-using-matrix-multiplications-1e8de8cd2544)

[Документация к ConvTranspose2d](
https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html?highlight=transpose#convtranspose2d)

```
torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, 
                         stride=1, padding=0, ...)
```
где:
* `in_channels`, `out_channels` - количество каналов в входной и выходной карте признаков,
* `kernel_size` - размер ядра свертки Transpose convolution, 
* `stride` - шаг свертки Transpose convolution,
* `padding` - размер отступов, устанавливаемых по краям входной карты признаков.



Пример использования

In [ ]:
input = torch.randn(1, 16, 16, 16) # define dummy input
print('Original size', input.shape)

downsample = nn.Conv2d(16, 16, 3, stride=2, padding=1) # define downsample layer
upsample = nn.ConvTranspose2d(16, 16, 3, stride=2, padding=1) # define upsample layer

# let`s downsample and upsample input
with torch.no_grad(): 
  output_1 = downsample(input)
  print("Downsampled size", output_1.size())

  output_2 = upsample(output_1, output_size = input.size())
  print("Upsampled size", output_2.size())

# plot results
fig, ax = plt.subplots(ncols=3, figsize=(15, 5), sharex=True, sharey=True)
sns.heatmap(input[0, 0, :, :], ax=ax[0], cbar=False, vmin=-2, vmax=2)
ax[0].set_title("Input")
sns.heatmap(output_1[0, 0, :, :], ax=ax[1], cbar=False, vmin=-2, vmax=2)
ax[1].set_title("Downsampled")
sns.heatmap(output_2[0, 0, :, :], ax=ax[2], cbar=False, vmin=-2, vmax=2)
ax[2].set_title("Upsampled")
plt.show()

## Пирамида признаков

При помощи Upsample мы можем сконструировать модель с нужной нам архитектурой:
<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/efficient_way_predict_pixel_class.png" width="800">

И она будет как-то работать, но возникает вопрос: не потеряется ли информация о мелких деталях изображения при передаче через центральный блок сети, где пространственное разрешение минимально. Такая проблема существует. 


Те, кто изучал классические методы машинного зрения, помнят, что  при извлечении дескрипторов особых точек([SIFT](https://en.wikipedia.org/wiki/Scale-invariant_feature_transform)) использовалась так называемая пирамида изображений.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/pyramid_of_images.png" width="650">

Идея состоит в последовательном уменьшении (масштабировании) изображения и последовательного извлечения признаков в разных разрешениях.


Эту идею можно использовать и при работе с картами признаков. При уменьшении пространственных размеров мы естественным образом получаем карты признаков с разным пространственным разрешением.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/pyramid_of_features.png" width="650">

Эти признаки можно сохранить и передать в разжимающие слои, где карты признаков будут иметь соответствующее пространственное разрешение:


<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/add_skip_connection.png" width="650">


Так же как и ResNet этот механизм носит название skip connection, но  признаки  не суммируются, а конкатенируются.







## U-Net: Convolutional Networks for Biomedical Image Segmentation

Рассмотренная нами схема используется в U-Net. Эта популярная модель для сегментации медицинских изображений изначально была предложена в статье [U-Net: Convolutional Networks for Biomedical Image Segmentation (Ronneberger et al., 2015)](https://arxiv.org/abs/1505.04597) для анализа  медицинских изображений.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/unet_scheme.png" width="700"></center>
<center><em>Архитектура U-Net (Ronneberger et al., 2015).</em></center>

[Реализация на PyTorch](https://github.com/milesial/Pytorch-UNet)

[U-Net на PyTorch Hub](https://pytorch.org/hub/mateuszbuda_brain-segmentation-pytorch_unet/)

[Блог-пост разбор](https://towardsdatascience.com/unet-line-by-line-explanation-9b191c76baf5)

Стоит обратить особое внимание на серые стрелки на схеме: они соответствуют операции конкатенации копий ранее полученных карт активаций, по аналогии с DenseNet. Чтобы это было возможно, необходимо поддерживать соответствие между размерами карт активаций в процессах снижения и повышения пространственных размерностей. Для этой цели, изменения размеров происходят только при операциях `MaxPool` и `MaxUnpool` - в обоих случаях, в два раза. 

В коде прямой проход может быть реализован, например, вот так:

```
def forward(self, x):
    out1 = self.block1(x) #  ------------------------------>
    out_pool1 = self.pool1(out1)

    out2 = self.block2(out_pool1)
    out_pool2 = self.pool2(out2)

    out3 = self.block3(out_pool2)
    out_pool3 = self.pool2(out3)

    out4 = self.block4(out_pool3)
    # return up
    out_up1 = self.up1(out4)

    out_cat1 = torch.cat((out_up1, out3), dim=1)
    out5 = self.block5(out_cat1)
    out_up2 = self.up2(out5)

    out_cat2 = torch.cat((out_up2, out2), dim=1)
    out6 = self.block6(out_cat2)
    out_up3 = self.up3(out6)

    out_cat3 = torch.cat((out_up3, out1), dim=1) # <-------
    out = self.block7(out_cat3)

    return out

```
После upsample блоков ReLU не используется.

## IoU - оценка точности

Как оценить качество предсказаний полученных от модели?

Базовой метрикой является Intersection over Union (IoU) она же коэффициент Жаккара([Jaccard index](https://en.wikipedia.org/wiki/Jaccard_index))

Имеется предсказание модели (фиолетовая маска) и целевая разметка  сделанная человеком (красная маска)*.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/iou_sample.png" width="400">

Необходимо оценить качество предсказания.




*Для простоты в примере маски прямоугольные, но та же логика будет работать  для масок произвольной формы.*


Метрика считается как отношение площади пересечения, к площади объединения двух масок: 

$$ IoU = \frac{|T \cap P|}{|T \cup P|} $$

T - True mask, P- predicted mask


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/iou_formula.png" width="400">

Если маски совпадут на 100%, то значение метрики будет равно 1 и это наилучший результат. При пустом пересечении IoU будет нулевым. Значения метрики лежат в интервале от [0..1].

В терминах ошибок первого/второго рода IoU можно записать как:

$$ IoU = \frac{TP}{TP + FP + FN} $$


TP - True positive == Пересечение (обозначено желтым)

FP - False Positive (остаток фиолетового прямоугольника)

FN - False Negative (остаток красного прямоугольника)



На базе этой метрики строится ряд производных от нее метрик, таких как Mean Average Precision, которую мы рассмотрим в разделе Детектирование.

Дополнительная информация: [Intersection over Union](http://datahacker.rs/deep-learning-intersection-over-union/)

## Loss функции для сегментации

Так как мы свели задачу сегментации к задаче классификации, то, казалось бы, надо использовать CrossEntropyLoss, с которой мы хорошо знакомы.



### Binary Cross-Entropy (BCE)
Но если предсказывается маска для объектов единственного класса, то задача сводится к бинарной классификации. Так как каждый канал на выходе последнего слоя выдает предсказание для единственного класса.

Это позволяет заменить softmax в CrossEntropyLoss на сигмоиду, а лосс функцию на бинарную кросс-энтропию (BCE)

In [ ]:
# https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
cross_entropy = nn.CrossEntropyLoss()

# https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html


one_class_out = torch.randn(1, 1, 32, 32)
one_class_target = torch.randn(1, 1, 32, 32)


bce_loss = nn.BCEWithLogitsLoss()
loss = bce_loss(one_class_out, one_class_target)
print("BCE", loss)


two_class_out = torch.randn(1, 2, 32, 32)
two_class_target = torch.randint(1, (1, 32, 32))

print("Model out for two class", two_class_out.shape)
print("Target for two class", two_class_target.shape)

loss = cross_entropy(two_class_out, two_class_target)

#bce_loss(two_class_out, two_class_target) throwing exception


print("Cross entropy loss", loss.item())

### Другие loss - функции для сегментации

#### Pixel-wise cross entropy loss


<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/pixel_wise_cross_entropy_loss.png" width="800">

#### DiceLoss

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/dice_loss.jpeg" width="800">

[Блог-пост про семантическую сегментацию](https://www.jeremyjordan.me/semantic-segmentation/)

In [ ]:
class BinaryDiceLoss(nn.Module):
    """Soft Dice loss of binary class
    Args:
        p: Denominator value: \sum{x^p} + \sum{y^p}, default: 2
        predict: A tensor of shape [N, *]
        target: A tensor of shape same with predict
       Returns:
        Loss tensor

    """

    def __init__(self, p=2, epsilon=1e-6):
        super().__init__()
        self.p = p  # pow degree
        self.epsilon = epsilon

    def forward(self, predict, target):
        predict = predict.flatten(1)
        target = target.flatten(1)

        # https://pytorch.org/docs/stable/generated/torch.mul.html
        num = torch.sum(torch.mul(predict, target), dim=1) + self.epsilon
        den = torch.sum(predict.pow(self.p) + target.pow(self.p), dim=1) + self.epsilon
        loss = 1 - 2 * num / den

        return loss.mean()  # over batch


criterion = BinaryDiceLoss()
output = torch.tensor([[[1, 1, 1], [1, 1, 1], [1, 1, 1]]], dtype=torch.float)


target = torch.tensor([[[1, 1, 1], [1, 1, 1], [1, 1, 1]]], dtype=torch.float)

soft_loss = criterion(output.unsqueeze(0), target.unsqueeze(0))
print("Loss", soft_loss)

## Fully Convolutional Networks

Сокращенно FCN для того, чтобы не было путаницы с Fully Connected Network, последние именуют MLP (Multi Layer Perceptron)

Примеры реализации: [1](https://pytorch.org/hub/pytorch_vision_fcn_resnet101/) и [2](https://pytorch.org/vision/stable/models.html#semantic-segmentation)


Предобученная модель была обучена на части датасета COCO train2017 (на 20 категориях, представленных так же в датасете  Pascal VOC). Использовались следующие классы:

`['__background__', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']`



https://arxiv.org/abs/1605.06211

[FCN — Fully Convolutional Network (Semantic Segmentation)](https://towardsdatascience.com/review-fcn-semantic-segmentation-eb8c9b50d2d1)

Архитектура похожа на Unet за тем исключением, что сеть не обязана быть симметричной.







<img src ="http://edunet.kea.su/repo/EduNet-web_dependencies/L12/fcn_backbone.png" width="500">

Такую сеть можно построить, взяв за основу другую сверточную архитектуру (*backbone*) , например `ResNet50` или `VGG16`.

<img src ="http://edunet.kea.su/repo/EduNet-web_dependencies/L12/fcn_changes.png" width="500">

И затем заменить полносвязанные  на свертки.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/fully_convolution_network_scheme.png" width="500">


В конце добавить `upsample` слой до нужных нам размеров. 


На вход такая модель может получать изображение произвольного размера. 
Так как для задач сегментации изменение размеров входного изображения приводит к потере важной информации о границах.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/fcn_1.png" width="1000">

сохранение информации с верхних слоев достигается не за счет конкатенации карт признаков как в UNet, а за счет суммы.




In [ ]:
import random

# fix seeds
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Compute on cpu or gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torchvision
from torchvision import transforms

def coco2pil(url):
    print(url)
    response = requests.get(url)
    return Image.open(BytesIO(response.content))


# load resnet50
fcn_model = torchvision.models.segmentation.fcn_resnet50(
    weights='FCN_ResNet50_Weights.DEFAULT', num_classes=21
)

classes = ["__background__", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor",
           ]

transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),  # ImageNet
    ]
)

pil_img = coco2pil("http://images.cocodataset.org/val2017/000000448263.jpg")
input_tensor = transform(pil_img)

with torch.no_grad():
    output = fcn_model(input_tensor.unsqueeze(0))  

Возвращаются 2 массива

* out - каждый пиксель отражает ненормированную вероятность, соответствующую предсказанию каждого класса.

* aux - содержит значения *auxillary loss* на пиксель. На инференсе `output['aux']` бесполезный.

In [ ]:
print("output keys: ", output.keys())  # Ordered dictionary
print("out: ", output["out"].shape, "Batch, class_num, h, w")
print("aux: ", output["aux"].shape, "Batch, class_num, h, w")

output_predictions = output["out"][0].argmax(0)  # for first element of batch
print(f'output_predictions: {output_predictions.shape}')

fig = plt.figure(figsize=(10, 10))
plt.imshow(pil_img)
plt.axis("off")
plt.show()

indexes = output_predictions

# plot all classes predictions
fig, ax = plt.subplots(nrows=4, ncols=5, figsize=(10, 10))
i = 0 # counter
for row in range(4):
    for col in range(5):
        mask = torch.zeros(indexes.shape)
        mask[indexes == i] = 255
        ax[row, col].set_title(classes[i])
        ax[row, col].imshow(mask)
        ax[row, col].axis("off")
        i += 1

plt.show()

## Обзор DeepLabv3+(2018)

DeepLab - семейство моделей для сегментации, значительно развивавшееся в течение четырёх лет. Основой данного рода моделей является использование **atros (dilated) convolutions** и, начиная со второй модели, **atros spatial pyramid pooling**, опирающейся на **spatial pyramid pooling**.

[Encoder-Decoder with Atrous Separable Convolution for Semantic Image Segmentation (Chen et al., 2018)](https://arxiv.org/abs/1802.02611v3)

[Реализация на PyTorch](https://pytorch.org/vision/stable/models.html#deeplabv3)

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/deeplabv3_scheme.png" width="800">

### Spatial pyramid pooling (SPP) layer

[Spatial Pyramid Pooling in Deep Convolutional Networks for Visual Recognition (He et al., 2014)](https://arxiv.org/abs/1406.4729)

**Spatial Pyramid Pooling (SPP)** - это *pooling* слой, который устраняет ограничение фиксированного размера сети, т.е. CNN не требует входного изображения фиксированного размера. В частности, мы добавляем слой SPP поверх последнего сверточного слоя. 

Слой SPP объединяет признаки и генерирует выходные данные фиксированной длины, которые затем поступают в MLP (или другие классификаторы). Другими словами, мы выполняем некоторую агрегацию информации на более глубоком этапе иерархии сети (между сверточными слоями и полностью связанными слоями), чтобы избежать необходимости обрезать или деформировать изображение в начале.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/spatial_pyramid_pooling_layer.png" width="700"></center>
<center><em>Схема SPP (He et al., 2014).</em></center>


В pytorch нет отдельного модуля, но результат может быть получен за счет применения нескольких AdaptiveMaxPool2d с разными размерами выходов.

### Atros (Dilated) Convolution

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/dilated_convolution.png" width="650">

**Dilated convolution** (расширенные свертки) - это тип свертки, который "раздувает" ядро, как-бы вставляя отверстия между элементами ядра. Дополнительный параметр (скорость расширения, **dilation**) указывает, насколько сильно расширяется ядро.


Фактически в такой свертке входные пикселы (признаки) участвуют через один (два, три ...)


```
torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, 
                padding=0, dilation=1, ...)
```
где:
* `in_channels`, `out_channels` - количество каналов в входной и выходной карте признаков,
* `kernel_size` - размер ядра свертки,
* `stride` - шаг свертки,
* `padding` - размер отступов, устанавливаемых по краям входной карты признаков,
* `dilation` - скорость расширения свертки.

[[doc] nn.Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)

In [ ]:
# for plot
def plot_conv2d(input, conv, output, dilation=1):
    fig, ax = plt.subplots(ncols=3, figsize=(15, 5), sharex=False, sharey=False)
    # input  
    sns.heatmap(input[0][0], ax=ax[0], annot=True, 
                fmt=".0f", cbar=False, vmin=0, vmax=20, linewidths=1)
    # kernel
    sns.heatmap(conv.weight.detach()[0, 0, :, :], ax=ax[1], annot=True, 
                fmt=".0f", cbar=False, vmin=0, vmax=20, linewidths=1)
    # output
    sns.heatmap(output[0, 0, :, :], ax=ax[2], annot=True, 
                fmt=".0f", cbar=False, vmin=0, vmax=20, linewidths=1)
    # titles
    ax[0].set_title("Input \nshape: " + str(input.shape))
    ax[1].set_title("Kernel \nshape: " + str(conv.weight.shape))
    ax[2].set_title("Output \nshape: " + str(output.shape))
    fig.suptitle("Dilation = " + str(dilation), y=1.05)

    plt.show()

In [ ]:
# Atros example
with torch.no_grad():
    # define dummy input
    input = torch.tensor([[[[1, 2, 3],
                            [1, 2, 3],
                            [1, 2, 3]]]], dtype=torch.float)
    
    # define conv layer, dilation = 1
    conv = nn.Conv2d(1, 1, kernel_size=2, dilation=1, bias=False)
    # define kernel weights
    conv.weight = nn.Parameter(torch.tensor([[[[1, 2],
                                               [2, 1]]]], dtype=torch.float))
    output = conv(input)
    plot_conv2d(input, conv, output, dilation=1)

In [ ]:
# change dilation to 2
with torch.no_grad():
    conv = nn.Conv2d(1, 1, kernel_size=2, dilation=2, bias=False)  # Fell free to change dilation
    conv.weight = nn.Parameter(torch.tensor([[[[1, 2],
                                               [2, 1]]]], dtype=torch.float))
    output = conv(input)
    plot_conv2d(input, conv, output, dilation=2)

In [ ]:
# change input tensor
with torch.no_grad():
    input = torch.tensor([[[[0, 1, 0],
                            [1, 1, 1],
                            [0, 1, 0]]]], dtype=torch.float)
    output = conv(input)
    plot_conv2d(input, conv, output, dilation=2)

# Детектирование (Object detection)


<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/detection
.png" width="400">

Детектирование - задача компьютерного зрения, в которой требуется
определить местоположение конкретных объектов на изображении.

При этом вычислять точные границы объектов не требуется, а достаточно определить только ограничивающие прямоугольники (bounding boxes), в которых находятся объекты.

В общем случае объекты могут принадлежать к различным классам, и объектов одного класса на изображении может быть несколько. 

## Детектирование единственного объекта

Но начнём с простой ситуации.

Пусть нас интересуют объекты только одного класса и мы знаем, что такой объект на изображении есть и он один. 

К примеру, мы разрабатываем систему по распознаванию документов:


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/predict_bounded_box_example.png" width="700">

На вход модели подаётся изображение, и предсказать требуется область, в которой объект локализован.
Область (bounding box) определяется набором координат вершин*. Собственно эти координаты и должна предсказать модель.





\* *Если наложить условие, что стороны многоугольника должны быть параллельны сторонам изображения, то можно ограничиться предсказанием 2-х координат.*







### Предсказание координат

Если задачу семантической сегментации получилось свести к классификации, то здесь будем использовать регрессию, поскольку предсказывать нужно не номер класса, а набор чисел.

В зависимости от требований эти числа могут нести разный смысл, например:

* координаты центра + ширина и высоту
* координаты правого верхнего и левого нижнего углов
* координаты вершин многоугольника ...

Но в любом случае задача остается регрессионной.


In [ ]:
# fix random_seed
import torch
import random
import numpy as np
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Compute on cpu or gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Решается она так:

Берем сверточную сеть, и меняем последний полносвязанный слой таким образом, чтобы количество  выходов совпадало с количеством координат, которые нам нужно предсказать.

Так для предсказания двух точек потребуется четыре выхода ( x1 , y1 , x2, y2 )




In [ ]:
from torchvision.models import resnet18
from torch import nn

# load pretrained model
resnet_detector = resnet18(weights='ResNet18_Weights.DEFAULT')

# Change "head" to predict coordinates (x1,y1 x2,y2)
resnet_detector.fc = nn.Linear(resnet_detector.fc.in_features, 4)  # x1,y1 x2,y2

Для обучения такой модели придется заменить функцию потерь на регрессионную, например MSE.

In [ ]:
criterion = nn.MSELoss()

# This is a random example. Don't expect good results
input = torch.rand((1, 3, 224, 224))
target = torch.tensor([[0.1, 0.1, 0.5, 0.5]])  # x1,y1 x2,y2 or x,y w,h
print(f'Target: {target}')
output = resnet_detector(input)
loss = criterion(output, target)
print(f'Output: {output}')
print(f'Loss: {loss}')

Координаты обычно предсказываются в процентах от длины и ширины изображения.
Таким образом, если bounding box целиком помещается на изображении, обе координаты будут находиться в интервале от $[0 .. 1]$




<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/predict_key_points.png" width="700"></center>
<center><em>Примеры предсказывания точек (Humphreys et al., 2020)</em></center>

[Recent Progress in Appearance-based Action Recognition (Humphreys et al., 2020)](https://arxiv.org/abs/2011.12619)


По такому принципу работают многие модели для поиска различных ключевых точек.
Например: на лице(facial landmarks) или теле человека.

### Multitask loss

Координаты прямоугольников мы предсказывать научились.

Теперь усложним задачу: объект остается один, но может принадлежать к различным классам. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/one_object.png" width="650">

То есть к задаче локализации добавляется классификация.

Задачу классификации мы умеем решать:

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/class_prediction.png" width="650">

Остается объединить классификацию с регрессией: 

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/multitask_loss_0.png" width="650">

Для этого нужно одновременно предсказывать:


*   вероятность принадлежности к классам
*   координаты ограничивающего прямогульника (bounding box)

Тогда выход последнего слоя будет иметь размер: 
$$N + 4$$


где N - количество классов (1000 для ImageNet), 

а 4 числа это координаты одного boundig box (x1,y1,x2,y2 или x,y,w,h)





**Как описать функцию потерь для такой модели?**

Можно суммировать loss для классификации и loss для регрессии.

$ L_{total} = L_{crossentropy}+L_{mse}$


<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/multitask_loss.png" width="650">







Однако значения разных loss могут иметь разные порядки, поэтому приходится подбирать весовые коэффициенты для каждого слагаемого. 
В общем случае функция потерь будет иметь вид:

$$L_{total} = \sum _iw_iL_i$$
где $w_i$ - весовые коэффициент каждой из лосс функций. 

Они являются гиперпараметрами модели и требуют подбора.

### Подбор весов для каждой компоненты loss




Можно подбирать веса компонентов loss в процессе обучения. Для этого к модели добавляется дополнительный слой:

[Multi-Task Learning Using Uncertainty to Weigh Losses for Scene Geometry and Semantics(Alex Kendall et al., 2018)](https://arxiv.org/abs/1705.07115)


[Пример реализации MultiTask learning](https://github.com/Hui-Li/multi-task-learning-example-PyTorch/blob/master/multi-task-learning-example-PyTorch.ipynb)

### Функции потерь для задач регрессии

**MAE vs MSE**

 <img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/regression_loss.png" width="700">

 $\displaystyle\mathrm{MSE} = \frac{\sum^n_{i=1}\left(y_i-y_i^p\right)^2}{n}$ - L2/ MSE/ Mean Squared Error/ Среднеквадратичная ошибка 


$\displaystyle\mathrm{MAE} = \frac{\sum^n_{i=1}\left|y_i-y_i^p\right|}{n}$ - L1/ MAE/ Mean Absolute Error/ Средняя ошибка


Huber vs Log-cos
 <img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/mae_vs_mse.png" width="700">

 $\displaystyle\
    L_{\delta}(y, f(x))=\left\{
                \begin{array}{ll}
                  \frac{1}{2}\left(y-f\left(x\right)\right)^2 \qquad &\mathrm{for}\  |y-f(x)| \leq \delta\\
                  \delta|y-f(x)|-\frac{1}{2}\delta^2 \qquad &\mathrm{otherwise}
                \end{array}
              \right.
  $ - Huber Loss/ Smooth Mean ABsolute Error/ Функция потерь Хьюбера


$\displaystyle L\left(y,y^p\right)=\sum_{i=1}^n log\left(cosh\left(y^p_i-y_i\right)\right)$ - Log-Cosh Loss/ Логарифм гиперболического косинуса

[5 Regression Loss Functions All Machine Learners Should Know](https://heartbeat.fritz.ai/5-regression-loss-functions-all-machine-learners-should-know-4fb140e9d4b0)

## Детектирование нескольких объектов

Как быть если объектов несколько?


Для каждого объекта нужно вернуть координаты(x1,y1,x2,y2) и класс (0 .. N). 
Соответственно количество выходов модели надо увеличивать...

Но нам неизвестно заранее, сколько объектов будет на изображении:


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/object_detection_multiple_object.gif" width="700">

[Stanford University CS231n: Detection and Segmentation](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture11.pdf)

### Наивный способ решения: скользящее окно

Одним из вариантов решения этой проблемы является применение классификатора ко всем возможным местоположениям объектов. Классификатор предсказывает, есть ли на выбранном фрагменте изображения один из интересующих нас объектов. Если нет - то фрагмент классифицируется как "фон". 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/naive_way_object_detection_multiple_object.gif" width="700">




Проблемой данного подхода является необходимость применять классификатор к огромному количеству различных фрагментов, что крайне дорого с точки зрения вычислений.

### Эвристика для поиска ROI

Вместо того чтобы применять классификатор "наобум", можно для начала выбрать те области изображения, в которых вероятность нахождения объекта наиболее высока и запускать классификатор лишь для них.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/heuristics_way_object_detection_multiple_object.png" width="700">

Такие области называются **regions of interest**, сокращённо - **ROI**. 

Для поиска таких областей можно использовать какой-либо эвристический алгоритм, например [Selective search](https://ivi.fnwi.uva.nl/isis/publications/bibtexbrowser.php?key=UijlingsIJCV2013&bib=all.bib).

#### Selective search



Selective search - известный алгоритмический метод поиска **ROI**.

Идея алгоритма состоит в разбиении изображения на небольшие области и последующего их итеративного объединения.


Объединение происходит на основании сходства, которое вычисляется как сумма 4-х метрик (см. иллюстрацию)




<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/selective_search.png" width="800">



[Статья про Selective Search](http://www.huppelen.nl/publications/selectiveSearchDraft.pdf)

Возвращает порядка 2000 прямоугольников для изображения, отрабатывает за несколько секунд на CPU.

### R-CNN (Region CNN)
Первая известная модель, построенная по описанному принципу:

- на изображении ищутся ROI 
- для каждого делается resize 
- каждый ROI обрабатывается сверточной сетью, которая предсказывает класс объекта, который в него попал




<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/region_of_interest_cnn.png" width="700">

Кроме класса модель предсказывает смещения для каждого bounding box

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/r_cnn_predict_bounded_box_shift.png" width="700">

### NMS

Теперь возникает другая проблема: в районе объекта алгоритм генерирует множество ограничивающих прямоугольников (bounding box), которые частично перекрывают друг друга.



<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/non_max_suppression.png" width="650">

Чтобы избавиться от них используется алгоритм
NMS (Non maxima suppression).

Его задача избавиться от bbox, которые накладаваются на истинный:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/non_max_suppression_pseudo_code.png" width="1000">

здесь $B$ - это массив всех bounding box,  $C$ - массив предсказаний модели относительно наличия объекта в соответствующем bounding box



Для оценки схожести обычно используется метрика IoU(same == IoU), а значение IoU ($\lambda_{nms}$), при котором bbox считаются принадлежащими одному объекту, является гиперпараметром (часто 0.5).


В Pytorch алгоритм NMS доступен в модуле torchvision.ops

`torchvision.ops.nms(boxes, scores, iou_threshold)`
где:
* `boxes` - массив bounding box,
* `scores` - предсказанная оценка,
* `iou_threshold` - порог IoU, NMS отбрасывает все перекрывающиеся поля с $IoU> iou\_threshold$

[[doc] torchvision.ops.nms](https://pytorch.org/vision/stable/generated/torchvision.ops.nms.html)

#### Soft NMS

Как видно из примера выше, NMS может удалять и истинные bounding box. 



Soft NMS не удаляет сомнительные bounding box а корректирует (понижает) для них score - вероятность (уверенность) в том, что в прямоугольнике действительно содержится объект.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/soft_non_max_suppression_pseudo_code.png" width="1000">


Результат:


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/soft_nms.png" width="500">




### Fast R-CNN




Проблемой R-CNN являлась скорость.
Так как мы вынуждены применять CNN порядка 2000 раз (в зависимости от эвристики, которая генерирует ROI)

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/r_cnn_scheme.png" width="800">

И решением, которое предложили авторы Fast R-CNN является поиск ROI не на самом изображении, а на карте признаков. В таком случае большая часть сверток выполняется только один раз.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/fast_r_cnn_scheme.png" width="700">

Это радикально ускоряет процесс

### ROI Pooling

Теперь появляется новая задача - получить карты признаков одинакового размера для всех ROI.

Для этого границы ROI проецируются на карту признаков.

Затем к полученным фрагментам карты признаков применяется операция max pooling и выходы получаются фиксированного размера. Теперь их можно подать на вход полносвязанного слоя.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/roi_pooling.png" width="650">

ROI pooling в Pytorch


```
torchvision.ops.roi_pool(input, boxes, output_size,...)
```
где:
* `input` -  тензор с входными картами признаков,
* `boxes` -  массив bounding box,
* `output_size` - размер вывода после ROI pooling.


[Документация Roi Pooling](https://pytorch.org/vision/stable/generated/torchvision.ops.roi_pool.html)

Статья: [Region of Interest Pooling](https://towardsdatascience.com/region-of-interest-pooling-f7c637f409af)


### ROI Align

Операция ROI pooling применялась в оригинальной модели Fast-RCNN. В дальнейшем она была заменена на Roi Align. Здесь признаки не отбрасываются, как это происходит при  max pooling, а их значения интерполируются на новую карту признаков.


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/roi_align.png" width="750">


Чтобы избежать квантования границ, RoIAlign использует билинейную интерполяцию для вычисления  значений входных признаков.





```
torchvision.ops.roi_align(input, boxes, output_size, ...)
```
где:
* `input` -  тензор с входными картами признаков,
* `boxes` -  массив bounding box,
* `output_size` - размер вывода после Roi Align.

[[doc] torchvision.ops.roi_align](https://pytorch.org/vision/stable/generated/torchvision.ops.roi_align.html)

[Understanding Region of Interest](https://towardsdatascience.com/understanding-region-of-interest-part-2-roi-align-and-roi-warp-f795196fc193)

### Faster R-CNN

**Идея: пусть сеть сама предсказывает ROI по карте признаков**



После того как в Fast-RCNN большая часть сверток стала запускаться единожды,
скорость работы нейросетевой части существенно возросла.



<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/compare_training_time_r_cnn_and_fast_r_cnn.png" width="900">

Теперь "узким местом" стала эвристика для поиска ROI.


Поэтому в следующей версии детектора (Faster R-CNN) от эвристики решено было избавиться, а  ROI искать при помощи дополнительной нейросети Region Proposal Network (RPN).




<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/faster_r_cnn_scheme.png" width="650">

Для обучения такой модели требуется посчитать четыре loss.


1. RPN классифицирует объект/не объект (классификация)
2. Координаты ROI предсказанные RPN (регрессия)
3. Класс объекта для каждого bounding box (классификация)
4. Координаты bounding boxes (регрессия)

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/faster_r_cnn_train_time.png" width="700">

В результате скорость увеличивается ещё почти в 10 раз, но для задач реального времени все равно остаётся неприемлемо низкой.

[Faster R-CNN на PyTorch](https://pytorch.org/vision/stable/models.html#faster-r-cnn)

In [ ]:
import torchvision
# load model
fr_rcnn_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
    weights='FasterRCNN_ResNet50_FPN_Weights.DEFAULT',
    progress=True,
    num_classes=91,
    weights_backbone='ResNet50_Weights.DEFAULT'
)
fr_rcnn_model.eval()

Загрузим данные

In [ ]:
from pycocotools.coco import COCO
import requests
import zipfile
import io

# load data
r = requests.get(
    "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()
coco = COCO("annotations/instances_val2017.json")

In [ ]:
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt 

def coco2pil(url):
    print(url)
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

catIds = coco.getCatIds(catNms=["person", "bicycle"]) # get category IDs
# person and bicycle
imgIds = coco.getImgIds(catIds=catIds)
img_list = coco.loadImgs(imgIds[12])  # http://images.cocodataset.org/val2017/000000370208.jpg
img = img_list[0]

# plot image
plt.figure(figsize=(10, 10))
pil_img = coco2pil(img["coco_url"])
plt.imshow(pil_img)

# plot boundy boxes
annIds = coco.getAnnIds(imgIds=img["id"])
anns = coco.loadAnns(annIds)
coco.showAnns(anns, draw_bbox=True)
plt.axis('off')
plt.show()
print('Image data: ')
img

In [ ]:
import torchvision.transforms.functional as TF
from PIL import ImageDraw

# lets predict objects by resnet50
with torch.no_grad():
    tensor = TF.pil_to_tensor(pil_img) / 255 # Normalize
    output = fr_rcnn_model(tensor.unsqueeze(0))
    draw = ImageDraw.Draw(pil_img)
    
    # plot rectangles
    for i, bbox in enumerate(output[0]["boxes"]):
        if output[0]["scores"][i] > 0.5: 
            draw.rectangle((tuple(bbox[:2].numpy()), tuple(bbox[2:].numpy())), width=2)

    plt.figure(figsize=(10, 10))
    plt.imshow(pil_img)
    plt.axis('off')
    plt.show()

#### Region proposal network (RPN)

Как устроена сеть предсказывающая ROI?




<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/roi_pooling.png" width="800">

Карта признаков имеет фиксированные и относительно небольшие пространственные размеры (например 20x15).



Поэтому можно вернуться к идее скользящего окна, которая была отвергнута в самом начале, из-за большого размера изображения.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/rpn_base.png" width="700">

Для карты признаков размером 20x15 количество ROI получится равным 3000, что сравнимо с количеством предсказаний производимых SelectiveSearch.






Далеко не всегда объект хорошо вписывается в квадрат:

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/rpn_aspect_ratio.png" width="900">

Поэтому для каждой точки на карте признаков (anchor) можно использовать окна нескольких форм:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/rpn_base_anchor.png" width="700">

Это позволит минимизировать корректировку* и лучше предсказывать ROI  для вытянутых объектов.


\* *помним, что для каждого прямоугольника предсказываются 4 числа обозначающих за сдвиг его вершин относительно начального положения.*

Для каждого окна предсказываются два значения:

* вероятность того, что в ROI находится объект (одно число)
* смещения (4 числа)

Сама сеть при этом может быть очень простой:

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/simple_nn_predict_objectness_and_boundary_box.png" width="400">

#### Two stage detector



Если присмотреться к схеме, можно заметить, что на среднем и верхнем уровнях выполняются схожие операции.


Разница в том, что на последнем слое предсказывается класс объекта, а на промежуточном только вероятность его присутствия (objectness). 

Корректировки для вершин bounding box предсказываются и в обоих случаях.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/two_stage_detector.png" width="750">

Можно сказать, что детектирование происходит в две стадии. 
Соответственно Faster RCNN == Two stage detector

### One Stage detector

Если сразу предсказывать класс, то можно избавиться от второй стадии.
В этом случае к списку классов нужно добавить еще один элемент, который заменит objectness, либо будет предсказанием класса "фон".

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/one_stage_detector.png" width="800">

Детекторы, которые работают "за один проход" быстрее, но потенциально менее точные.



<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/yolo_ssd_retinanet.png" width="1000">

Рассмотрим несколько моделей, построенных по этому принципу:
YOLO, SSD, RetinaNet

[Сравнение скорости моделей](https://pytorch.org/vision/stable/models.html#runtime-characteristics)

#### SSD: Single Shot MultiBox Detector

[SSD: Single Shot MultiBox Detector (Liu et al., 2015)](https://arxiv.org/abs/1512.02325)

<img src ="http://edunet.kea.su/repo/EduNet-web_dependencies/L12/ssd_default_boxes.png" width="700">



* Кандидаты в ROI (default box) выбираются на нескольких слоях (4,7, 8, 9,11)
* Количество форм окон (default box) на картах признаков зависит от слоя: от 4 до 6 




<img src ="http://edunet.kea.su/repo/EduNet-web_dependencies/L12/single_shot_multibox_detector_scheme.png" width="1500">

* В качестве backbone используется VGG-16, предобученная на ImageNet
* Добавлен класс для "background"


В общей сложности делается 8732 предсказаний, каждое содержит 4 + (N + 1) чисел. 

N - это количество классов без фона.
4 - смещения


[Подробнее.](https://towardsdatascience.com/review-ssd-single-shot-detector-object-detection-851a94607d11)

### Loss для детектора

Как подсчитать loss для детектора. Теоретически понятно, что она должна включать в себя две части: ошибку локализации и ошибку классификации.

И для SSD лосс так и выглядит:

$$L(x,c,l,g) = \frac{1}{N}(L_{conf}(x,c) + \alpha L_{loc}(x,l,g))$$


Однако если мы будем считать ошибку локализации для всех default box, то она никогда не будет нулевой.
Default box очень плотно перекрывает все изображение и в большинство из них объект не попадет, особенно если объект один и небольшой.









<img src ="http://edunet.kea.su/repo/EduNet-web_dependencies/L12/default_boxes.png" width="700">

Поэтому часть default box при подсчете лосс игнорируются. Используются только те, у которых  большая площадь пересечения с одним из истинным bounding box больше порога (IoU > 0.5)



$$L(x,l,g)_{loc} = \sum_{i \in Pos}^{N} x_{i,j}^{k}smooth_{L1}(l_i, g_j)$$


Здесь:

$l$ - финальные координаты, предсказанного bounding box, с учетом смещений

$g$ - координаты истинного bounding box

$M$ - количество истинных (ground true) bounding box - ов

$Pos$ - список отобранных default box пересекающихся с истинными

$x_{i,j}^{k} = \{1,0\}$ - индикатор того что комбинация default и box - валидна.



> i - индекс default box
> j - индекс истинного (ground true) bounding box
> p - номер класса, к которому относится ground true bounding box (не степень)

$smooth_{L1}$ - [Комбинация L1 и L2](https://pytorch.org/docs/stable/generated/torch.nn.SmoothL1Loss.html) 

Компонент, отвечающий за локализацию:

$$L(x,l,g)_{conf} = -\sum_{i \in Pos} x_{i,j}^{k} log(softmax(c_{i}^{p})) -\sum_{i \in Neg} log(softmax(c_{i}^{0}))$$

$c_{i}^{p}$ - вектор score для i-того default box, p - номер истинного класса, соответствующего bounding box из разметки

$Pos$ - список отобранных default box, не пересекающихся с истинными (IoU < treshold)






\* *Формулы лоя лосс осознанно упрощены. Например, мы опустили расчет L1 для смещений, что является технической деталью.*






#### FocalLoss

Следующий заслуживающий внимания one-stage детектор это Retina Net - [Focal Loss for Dense Object Detection (Lin et al., 2017)](https://arxiv.org/abs/1708.02002)

Собственно авторы придумали новую лосс - функцию (Focal Loss) и опубликовали модель, чтобы продемонстрировать её эффективность.

Чтобы понять какую проблему решает FocalLoss, давайте посмотрим на второй компонент Loss классификации для SSD:

$$L_{conf} =  \ ...\  -\sum_{i \in Neg} log(softmax(c_{i}^{0}))$$

Это кросс - энтропия для bounding box, содержащих фон. Тут нет ошибки: когда модель обучится правильно предсказывать класс фона (background) каждая из этих компонент будет небольшой.

Проблема в том, что таких компонент очень много. Детектор предсказывает несколько тысяч, или десятков тысяч bounding box. Подавляющая часть из них приходится на фон. Cумма большого количества этих небольших лосс становится заметным числом и мешает учиться классифицировать реальные объекты.

Как решается эта проблема в Focal loss?


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/focal_loss_vs_ce.png" width="700">





Фактически лосс для уверенно классифицированных объектов дополнительно занижается. Это похоже на взвешивание при дисбалансе классов.

Достигается этот эффект путем домножения на коэффициент: $ (1-p_{t})^\gamma$

Здесь:

$ p_{t} $ - вероятность истинного класса

$ \gamma $ - число большее 1 и являющееся гиперпараметром


Пока модель ошибается, $p_{t}$ - мало, и выражение в скобках соответственно близко к 1-це 

Когда модель обучилась, $p_{t}$ становится близким к 1-це, разность в скобках становится маленьким числом, которое при возведении в степень > 1. Таким образом, домножение на это небольшое число нивелирует вклад верно классифицированных объектов.

Это позволяет модели сосредоточиться на изучении сложных объектов (hard example )





[Блог-пост: Что такое Focal Loss и когда его использовать](https://amaarora.github.io/2020/06/29/FocalLoss.html)






#### Feature pyramid network


Вторым полезным нововведением  в retina-net стало использование пирамиды признаков.

[Feature Pyramid Networks for Object Detection (Sergelius et al., 2016)](https://arxiv.org/abs/1612.0314)



<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/retinanet_use_outputs_fpn.png" width="900">


RetinaNet использует выходы FPN и для предсказаний класса и bbox. Мы уже обсуждали пирамиды признаков применительно к сетям для сегментации, в частности FCN. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/pyramid_of_features.png" width="650">

На каждом сверточном слое извлекаются карты признаков. 

Их пространственное разрешение постепенно уменьшается, а глубина (количество каналов) увеличивается.


Но первые слои содержат мало семантической информации (только низкоуровневые признаки).  А карты признаков с глубоких слоев имеют низкое пространственное разрешение, что не позволяет качественно определить границы объектов.





<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/semantic_information.png" width="650">

Так же, как и в случае с сегментацией, точность повышается, если делать предсказания на картах содержащих признаки для разных масштабов.


При этом можно получать карты с большим пространственным разрешением не просто сохраняя их в памяти, а еще и прибавляя к ним значения признаков с более глубоких слоев, предварительно интерполировав их (Upsample).

Идея состоит в том, чтобы делать предсказание с учетом семантической информации, полученной на более глубоких слоях. Здесь признаки   суммируются, а не конкатенируются.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/resnet_scheme.png" width="650">

Затем к новым картам признаков может применяться дополнительная свертка.

На выходе получаем карты признаков P2 - P5 на которых уже предсказываются bounding box.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/resnet_prediction_head_scheme.png" width="850">

В случае 2-stage детектора (RCNN) новые карты признаков подаются на вход RPN.

<img src ="http://edunet.kea.su/repo/EduNet-web_dependencies/L12/features_from_blackbone.jpeg" width="1100">

А признаки для предсказаний используются из backbone 

Дополнительно: [Блог-пост про FPN](https://jonathan-hui.medium.com/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c)

### YOLO

* [You Only Look Once: Unified, Real-Time Object Detection (Redmon et. al., 2015)](https://arxiv.org/abs/1506.02640) 
* [YOLO9000: Better, Faster, Stronger (Redmon et. al., 2015)](https://arxiv.org/abs/1612.08242)
* [YOLOv3: An Incremental Improvement (Redmon et. al., 2018)](https://arxiv.org/abs/1804.02767)
* [YOLOv4: Optimal Speed and Accuracy of Object Detection (Bochkovskiy et al., 2020)](https://arxiv.org/abs/2004.10934)
* Июнь 2020. [YOLOv5 (Glenn Jocher)](https://github.com/ultralytics/yolov5)
* Июль 2021. [YOLOX: Exceeding YOLO Series in 2021 (Ge et al., 2021)](https://arxiv.org/abs/2107.08430)

Первая версия YOLO вышла в том же году что и SSD. На тот момет, детектор несколько проигрывал SSD в точности.

Однако благодаря усилиям Joseph Redmon проект поддерживался и развивался в течение нескольких лет.


 3-я версия детектора оказалась настолько удачной, что до сих пор о ней можно прочесть:  "YOLOv3, one of the most widely used detectors in industry" [2021](https://arxiv.org/abs/2107.08430)




В настоящий момент можно сказать, что YOLO - это  оптимальный детектор по соотношению качество распознавания/скорость.



#### YOLOv3

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/yolov3.png" width="1200">


В качестве backbone используется оригинальная сверточная сеть Darknet53, задействующая слои Batch Norm и Skip connection. Но есть реализации с ResNet в качестве backbone.

Детектор использует большинство техник, которые мы обсудили:

* Default boxes извлекаются на трех слоях различной глубины. Для каждой ячейки  предсказывается З окна
* FPN: признаки конкатенируются, а не складываются
* Resolution augmentation: При обучении разрешение входных изображений менялось (10 input resolution steps between 384x384 and 672x672)
* В качестве лосс для классификации используется бинарная кросс-энтропия, позволяющая предсказывать несколько объектов в одном bounding box. Что позволяет использовать детектор с multilabel датасетами, где один объект может иметь несколько меток (person & woman)
* Предсказывается дополнительный параметр objectness score. Он не связан с классификацией. Его задача предсказать насколько вероятно, что в предсказанном default boх действительно есть объект и он будет учитываться при подсчете loss



<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/yolov3_prediction.png" width="400">

Соответственно, для каждого anchor box предсказывается вот такой вектор значений:

* смещения
* objectness вероятность наличия объекта 
* scores - уверенность того, что bbox содержит объект определенного класса. Для моделей тренированных на СOCO классов 80

Для 80-ти классов получается 85 значений на один default box

#### Предсказание смещений

Эксперименты показывают (см. текст статьи по YOLOv3), что предсказывать абсолютные значения смещений неэффективно. 

Для примера можно взглянуть, как преобразуются предсказания YOLOv3 для получения финальных координат bounding box

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/yolov3_coordinates_prediction.png" width="500">



$c_{x} , c_{y}$ - это координаты центра default box
$c_{w} , c_{h}$ - это ширина и высота default box
$t_{x} , t_{y}$ - предсказанные смещения  для центра
$t_{w} , t_{h}$ - предсказанные корректировки  для ширины и высоты

$b_{x} , b_{y}, b_{w}, b_{h}$ - координаты центра, ширина и высота финального предсказанного bouning box. Значения в процентах от ширины и длины исходного изображения

$σ(x) $ - сигмоида

$e$ - число Эйлера




[YOLO v3: Better, not Faster, Stronger](https://towardsdatascience.com/yolo-v3-object-detection-53fb7d3bfe6b#:~:text=YOLO%20v2%20used%20a%20custom,more%20layers%20for%20object%20detection.&text=First%2C%20YOLO%20v3%20uses%20a,layer%20network%20trained%20on%20Imagenet.)

#### YOLOv4

YOLOv3 на момент выхода стал одним из самых быстрых детекторов и последней версией за авторством Джозефа Редмона.

YOLOv4 это детище других авторов, модель не стала быстрее, но стала намного более точной.



<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/yolov4.jpeg" width="700">

[YOLO v4](https://medium.com/visionwizard/yolov4-version-3-proposed-workflow-e4fa175b902)

Что же добавили авторы:

1. Поменялся Backbone:
* Увеличилось количество слоев
* Добавился SPP block
* Добавились Dense в блоки ...

2. Жесткая аугментация

* В том числе Mosaic,  и online Augmentation


##### Mosaic augmentation

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/mosaic_augmentation.jpg" width="900">

Изображение, которое подается на вход сети, склеивается из нескольких (4) фрагментов разных изображений. При этом статистика для Batch norm считается по 4-м полным изображениям.
Такая стратегия позволяет уменьшить размер batch, что важно при работе с изображениями, имеющими большое разрешение.

 Path Aggregation Network Module(PAN)

Self-Adversarial Training (SAT)

#### YOLOv5


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/yolov5.png" width="700">


Статья не публиковалась.
Точность сравнима  с v4, но модель определенно лучше упакована.

In [ ]:
catIds = coco.getCatIds(catNms=["person", "bicycle"])
# person and bicycle
imgIds = coco.getImgIds(catIds=catIds)
img_list = coco.loadImgs(imgIds[5])
img = img_list[0]

pil_img = coco2pil(img["coco_url"])
plt.figure(figsize=(10, 10))
plt.imshow(pil_img)

annIds = coco.getAnnIds(imgIds=img["id"])
anns = coco.loadAnns(annIds)
coco.showAnns(anns, draw_bbox=True)
plt.axis('off')
plt.show()
print("Image data:")
img

Загрузка модели с Torch Hub



In [ ]:
from IPython.display import clear_output
# Load model from torch
model = torch.hub.load("ultralytics/yolov5", "yolov5s", pretrained=True, trust_repo=True)
clear_output()

Из коробки работает с изображениями в разных форматах и даже url, автоматически меняет размер входного изображения, возвращает объект с результатами ...

In [ ]:
# Apply yolov5 model
results = model(pil_img)
results.print() # print predicted results
results.save()  # image on disk

print(f'\nresults.xyxy type: {type(results.xyxy)}\
\nlen(results.xyxy): {len(results.xyxy)}\
\nresults.xyxy[0].shape: {results.xyxy[0].shape}')

results.pandas().xyxy[0]

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

# plot predicted results
cv_img = np.array(pil_img)
RGB_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)

annos = results.pandas().xyxy[0]

for i in range(len(annos)):
    x_min, y_min, x_max, y_max = (
        int(annos["xmin"].iloc[i]),
        int(annos["ymin"].iloc[i]),
        int(annos["xmax"].iloc[i]),
        int(annos["ymax"].iloc[i]),
    )

    if annos["name"].iloc[i] == "person":
        color = (255, 255, 255)
    if annos["name"].iloc[i] == "bicycle":
        color = (0, 0, 255)
    if annos["name"].iloc[i] == "backpack":
        color = (0, 255, 0)
    RGB_img = cv2.rectangle(RGB_img, (x_min, y_min), (x_max, y_max), color, 2)
cv2_imshow(RGB_img)

Однако если подать на вход модели тензор, выходы радикально меняются. YOLO переходит в режим обучения, что довольно не очевидно

In [ ]:
input = torch.rand((1, 3, 416, 416))
results = model(input)

print(f'type(results): {type(results)}\nlen(results): {len(results)}\n')
print(f'results[0].shape: {results[0].shape}\n')

#### YOLOX

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/yolox_speed.png" width="500">


[YOLOX: Exceeding YOLO Series in 2021](https://arxiv.org/abs/2107.08430)

Авторы использовали в качестве baseline модели YOLOv3, и, убедившись на ней в эффективности усовершенствований, применили некоторые из них к YOLOv4 и YOLOv5.

Далее рассмотрим список нововведений.

#### Decoupled head

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/yolox_decoupled_head.png" width="700">

Так же как в RetinaNet для регрессии и классификации используются различные головы (подсети).

#### Anchor-free

Все детекторы, которые рассматривались ранее, использовали несколько предопределенных default box (anchor) для каждой точки на карте признаков.
Количество и размер этих якорных окон являются гиперпараметрами модели.


В 2019 г вышла статья [FCOS: Fully Convolutional One-Stage](https://towardsdatascience.com/forget-the-hassles-of-anchor-boxes-with-fcos-fully-convolutional-one-stage-object-detection-fc0e25622e1c) где авторы отказываются от такого подхода.  Для каждой точки на карте признаков сразу предсказывают один bounding box.

Если пиксель соответствующий центру предсказанного bounding box попадает в истинный (ground true) bounding box, то он используется при подсчете loss.


Это подход был применен и в YOLOX.




 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/yolox_techniques.png" width="1000">




* decoupled head
* anchor-free, 
* and advanced label assigning strategy (SimOTA)
* MultiPositives
* OTA/SimOTA

## Нard Example Mining

При обучении модели, мы можем обнаружить, что средняя ошибка на всех данных достаточно маленькая, однако, ошибка на редких нетипичных данных довольно высока. При этом, нетипичные данные необязательно являются выбросами.

Разберемся, почему так происходит:
К примеру, рассмотрим задачу обнаружения автомобилей на потоках данных с камер наружного видеонаблюдения. Если в обучающем наборе большая часть данных - снимки, сделанные днём, то качество работы модели ночью будет низким. В данном случае, "нетипичными" данными будут ночные снимки. Но, на самом-то деле, "нетипичных" случаев может быть довольно много, и некоторые из них могут происходить даже днём: 
* изменение погоды (изменение яркости, резкости, помехи на изображении)
* смена сезона (снег либо листья могут покрыть дорогу - изменение фона)
* машины с экзотическими узорами на кузове

Довольно простым и эффективным решением проблемы является сбор "сложных" случаев (**hard example mining**) и дообучение модели на них. При этом, поскольку модель уже довольно хорошо работает на большей части данных, можно дополнительно удалить часть данных из обучающей выборки - таким образом, мы сосредотачиваем модель на обучении на сложных примерах.

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/hard_example_mining.png" width="1000">

### Online hard example mining

В некоторых случаях, hard exapmle mining можно выполнять прямо во время формирования батча, "на лету". В таких случаях, говорят про **online hard example mining**.

Один из вариантов может быть реализован в two-stage детекторах.
Напоминаю: первая часть детектора отвечает за обнаружение regions of interest (RoI), затем выполняется (как правило, сравнительно вычислительно дешёвая) классификация. Одним из вариантов реализации идеи может быть выполнение forward pass классификатора по всем предложенным RoI, и затем формирование батча, в котором будет выделено определённое количество "мест" под RoI, предсказания на которых выполняются наихудшим образом.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/online_hard_example_mining.png" width="700">

### Focal loss

Одним из интересных способов борьбы с падением качества работы модели на нестандартных данных, является уже упомянутый **focal loss**.
При использовании кросс-энтропии, loss от большого количества "хорошо распознанных" примеров суммарно может быть более значительным, чем loss от малого количества "плохо распознанных" примеров. Итого, при обучении модель стремится снизить свою ошибку на большинстве и так уже неплохо предсказываемых примеров, не стремясь исправлять высокую ошибку на редких примерах.

Идея **focal loss** заключается в том, что можно попробовать снизить значения **cross-entropy loss** на и так уже неплохо предсказываемых примерах, чтобы дать модели возможность лучше обучиться на "сложных" для неё примерах.

Давайте посчитаем для различных значений $γ$, сколько понадобится примеров с небольшой ошибкой, чтобы получить суммарный **focal loss** примерно такой же, как у одного примера с большой ошибкой.

In [ ]:
def cross_entropy(percent_correct):
    return -np.log(percent_correct)

def focal_loss(percent_correct, gamma=5):
    return cross_entropy(percent_correct) * (1 - percent_correct) ** gamma

p1 = 0.8  # probability for correct examples predictions
p2 = 0.4  # probability for hard examples predictions
gammas = [0, 1, 5, 9, 13, 17]

print(f"For probability for correct examples predictions {p1} and probability for hard examples predictions {p2}\n")

for gamma in gammas:
    fl1 = focal_loss(p1, gamma)
    fl2 = focal_loss(p2, gamma)

    print(f"gamma = {gamma},".ljust(15), \
          f"for an equal loss with a problematic prediction, almost correct ones are required {int(fl2 / fl1)}")

Как видно, при увеличении значения $\gamma$, можно достичь значительного роста "важности" примеров с высокой ошибкой, что, по сути, позволяет модели обращать внимание на "hard examples".

[Блог пост про Hard Mining Example](https://erogol.com/online-hard-example-mining-pytorch/)

[Training Region-based Object Detectors with Online Hard Example Mining (Shrivastava et al., 2016)](https://arxiv.org/abs/1604.03540)

[Loss Rank Mining: A General Hard ExampleMining Method for Real-time Detectors (Yu et al., 2018)](https://arxiv.org/abs/1804.04606)

# Instance Segmentation

<img src ="https://edunet.kea.su/repo/EduNet-content/L12/out/detection_instance_segmentation.png" width="650">

[COCO panoptic](https://cocodataset.org/#panoptic-2020)

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/panoptic_segmentation.png" width="700">

**Mask R-CNN** (Detectron) - концептуально простая, гибкая и общая схема сегментации объектов. Подход эффективно обнаруживает объекты на изображении и одновременно генерирует высококачественную маску сегментации для каждого объекта. 

Метод, названный Mask R-CNN, расширяет Faster R-CNN (который мы обсуждали ранее), добавляя ветвь для предсказания маски объекта параллельно с существующей ветвью для распознавания *bounding boxes*. 

Код доступен [по ссылке](https://github.com/facebookresearch/Detectron)

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/mask_r_cnn.png" width="750">

[Модель Mask R-CNN](https://pytorch.org/vision/stable/models.html#mask-r-cnn)

[Пример запуска Mask R-CNN есть в документации Pytorch](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html)



# Оценка качества детекции

## mAP - mean Average Precision

AP (*Average Precision* - средняя точность) - это популярная метрика для измерения качества детекторов объектов, таких как Faster R-CNN, SSD и др. Средняя точность вычисляет среднее значение *precision* (точности) для значения *recall* от 0 до 1. Звучит сложно, но на самом деле довольно просто. Давайте разберем на конкретных примерах. Но перед этим давайте вкратце разберем, что такое *precision*, *recall* и IoU.

### Precision & recall

**Precision** измеряет, насколько точны предсказания сети (т.е. процент правильных предсказаний)

**Recall** измеряет, насколько хорошо сеть находит все положительные срабатывания (*positives*). Например, мы можем найти 80% возможных положительных срабатываний в наших K лучших предсказаниях.

Вот их математические определения:

$\displaystyle\mathrm{Precision} = \frac{TP}{TP+FP}$

$\displaystyle\mathrm{Recall} = \frac{TP}{TP+FN}$

где $TP$ - True Positive, $TN$ - True Negative, $FP$ - False Positive, $FN$ - False Negative.



Например, мы пытаемся детектировать яблоки на фотографиях. Предположим, мы обработали 20 фотографий (на 10 фотографиях по одному яблоку и на 10 фотографиях яблок нет) и обнаружили что:

* в 7 случаях наша нейросеть обнаружила яблоко там, где оно было на самом деле (True Positive)
* в 3 случаях не обнаружила яблоко там, где оно было (False Negative)
* в 4 случаях обнаружила яблоко там, где его не было (False Positive)
* в 6 случаях правильно определила, что на фотографии яблок нет (True Negative)



Посчитаем precision и recall:

In [ ]:
def precision(TP, FP):
    return TP / (TP + FP)

def recall(TP, FN):
    return TP / (TP + FN)

pres = precision(TP=7, FP=4)
rec  = recall(TP=7, FN=3)

print('Precision = %.2f' % pres)
print('Recall = %.2f' % rec)

### IoU (Intersection over union)

IoU измеряет перекрытие между двумя границами. Мы используем его для измерения того, насколько сильно наша предсказанная граница совпадает с истиной (границей реального объекта). В некоторых наборах данных мы заранее определяем порог IoU (например, 0.5) для классификации того, является ли предсказание True Positive или False Positive.

Например, рассмотрим предсказание сети для фотографии яблока:

In [ ]:
!wget -q https://edunet.kea.su/repo/EduNet-web_dependencies/L12/red_apple.jpg -O img.jpg

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

img = Image.open('img.jpg')
array = np.array(img)

fig,ax = plt.subplots(figsize=(5, 5))

x0 = 75
y0 = 80
w0 = 450
h0 = 440

ground_truth = Rectangle((x0, y0), w0, h0, linewidth=2, edgecolor='g', linestyle='--', facecolor='none')

x1 = 90
y1 = 120
w1 = 500
h1 = 310
predicted = Rectangle((x1, y1), w1, h1, linewidth=2, edgecolor='b', facecolor='none')

ax.add_patch(ground_truth)
ax.add_patch(predicted)

ax.imshow(array)
ax.axis('off')
plt.show()

IoU определена как

$\displaystyle\mathrm{IoU} = \frac{\text{area of overlap}}{\text{area of union}}$

Посчитаем *area of overlap* и *area of union*:

In [ ]:
def area_of_overlap(x0, y0, x1, y1, w0, w1, h0, h1):
    x0_max = x0 + w0
    y0_max = y0 + h0
    x1_max = x1 + w1
    y1_max = y1 + h1
    dx = min(x0_max, x1_max) - max(x0, x1)
    dy = min(y0_max, y1_max) - max(y0, y1)
    if (dx >= 0) and (dy >= 0):
        return dx * dy

def area_of_rectangle(w, h):
    return w * h
# Compute intersection areas of the predictions
a_of_overlap = area_of_overlap(x0, y0, x1, y1, w0, w1, h0, h1)

# Compute individual areas of the rectangles
a_0 = area_of_rectangle(w0, h0)
a_1 = area_of_rectangle(w1, h1)

# Compute area of their union
a_of_union = a_0 + a_1 - a_of_overlap

print('Area of overlap = %i' % a_of_overlap)
print('Area of union = %i' % a_of_union)

Теперь посчитаем IoU

In [ ]:
IoU = a_of_overlap / a_of_union
print('IoU = %.2f' % IoU)

Посмотрим, как будет меняться IoU в зависимости от качества предсказания

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets

def plot_predictions_and_calculate_IoU(x1, y1, w1, h1):
    fig,ax = plt.subplots(figsize=(5, 5))

    x0 = 75
    y0 = 80
    w0 = 450
    h0 = 440

    ground_truth = Rectangle((x0, y0), w0, h0, linewidth=2, edgecolor='g', linestyle='--', facecolor='none')
    predicted = Rectangle((x1, y1), w1, h1, linewidth=2, edgecolor='b', facecolor='none')

    ax.add_patch(ground_truth)
    ax.add_patch(predicted)

    ax.imshow(array)
    ax.axis('off');

    a_of_overlap = area_of_overlap(x0, y0, x1, y1, w0, w1, h0, h1)
    a_0 = area_of_rectangle(w0, h0)
    a_1 = area_of_rectangle(w1, h1)
    a_of_union = a_0 + a_1 - a_of_overlap
    IoU = a_of_overlap / a_of_union
    print('IoU = %.2f' % IoU)
    plt.show()

interact(plot_predictions_and_calculate_IoU, 
         x1 = widgets.IntSlider(min=0, max=array.shape[0], step=10, value=90), 
         y1 = widgets.IntSlider(min=0, max=array.shape[1], step=10, value=120), 
         w1 = widgets.IntSlider(min=0, max=array.shape[0], step=10, value=500),
         h1 = widgets.IntSlider(min=0, max=array.shape[1], step=10, value=310)
        );

Видим, что чем лучше предсказание совпадает с реальностью - тем выше у нас значение метрики IoU

### Average Precision

Продолжая тему яблок, возьмем все те же 20 фотографий на 10 из которых яблоки были, а на 10 яблок не было. И представим, что у нас есть некая нейросеть, которая выдает следующие предсказания по всему датасету (или например по батчу):

In [ ]:
import pandas as pd

nn_preds = pd.DataFrame({'IoU' : [0.6, 0.98, 0.4, 0.3, 0.1, 0.96, 0.7, 0.3, 0.2, 0.8],  
                        'precision' : [1,1,0.67,0.5,0.4,0.5,0.57,0.5,0.44,0.5],
                        'recall' : [0.2,0.4,0.4,0.4,0.4,0.6,0.8,0.8,0.8,1]})
nn_preds

Будем считать, что если $\mathrm{IoU} \geq 0.5$ - то предсказание правильное

In [ ]:
nn_preds['correct'] = False
nn_preds.loc[nn_preds['IoU'] >= 0.5, 'correct'] = True

nn_preds

Можно заметить, что precision имеет зигзагообразный характер - она снижается при ложных срабатываниях и снова повышается при истинных срабатываниях. 

Давайте построим график precision от recall и убедимся

In [ ]:
fig,ax = plt.subplots(figsize=(10, 3))
ax.plot(nn_preds.recall, nn_preds.precision)
ax.grid('on')
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
plt.show()

По определению, чтобы найти AP, нужно найти площадь под кривой recall-precision:

$\displaystyle AP = \int_0^1p(r)dr$



Precision и recall всегда находятся в пределах от 0 до 1. Поэтому $AP$ также находится в пределах от 0 до 1. Перед расчетом $AP$ для обнаружения объекта мы часто сначала сглаживаем зигзагообразный рисунок (на каждом уровне recall мы заменяем каждое значение precision максимальным значением точности справа от этого уровня отзыва)

In [ ]:
def smooth_precision(nn_preds):
    smooth_prec = []
    for i in range(len(nn_preds.precision.values)):
        max = nn_preds.precision.values[i:].max()
        smooth_prec.append(max)
    nn_preds['smooth_precision'] = smooth_prec
    return nn_preds

nn_preds = smooth_precision(nn_preds)

Давайте посмотрим, как это выглядит на графике

In [ ]:
fig,ax = plt.subplots(figsize=(10, 3))
ax.plot(nn_preds.recall, nn_preds.precision, label='precision', color='blue')
ax.plot(nn_preds.recall, nn_preds.smooth_precision, label='smooth precision', color='red')
ax.grid('on')
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.legend()
plt.show()

Зачем нам нужно сглаживание? Чтобы снизить влияние случайных выбросов и "прыжков" в предсказаниях модели

Теперь давайте посчитаем-таки $AP$

In [ ]:
fig,ax = plt.subplots(figsize=(10, 3))
ax.plot(nn_preds.recall, nn_preds.precision, label='precision', color='blue')
ax.plot(nn_preds.recall, nn_preds.smooth_precision, label='smooth precision', color='red')
ax.fill_between(nn_preds.recall, nn_preds.smooth_precision, 
                np.zeros_like(nn_preds.smooth_precision), 
                color='red', alpha=0.1,
                label='Area under the curve')
ax.grid('on')
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_ylim(0.35, 1.05)
ax.legend()
plt.show()

In [ ]:
from sklearn.metrics import auc

AP = auc(nn_preds.recall, nn_preds.smooth_precision)

print('AP = %.2f' % AP)

### COCO mAP

В последних исследовательских работах, как правило, приводятся результаты только для набора данных COCO. Для COCO AP - это среднее значение (*mean*) по нескольким IoU (минимальный IoU, который следует считать положительным совпадением). AP@[.5:.95] соответствует среднему AP для IoU от 0.5 до 0.95 с шагом 0.05. 

Давайте попробуем посчитать mAP. Для этого посчитаем AP для каждого уровня IoU:

In [ ]:
nn_prediction_at_iou = []
APs = []
for iou in np.arange(0.5, 1, 0.05):
    nn_preds_limited = nn_preds[nn_preds['IoU'] >= iou].copy()
    nn_preds_limited = smooth_precision(nn_preds_limited)
    AP = auc(nn_preds_limited.recall, nn_preds_limited.smooth_precision)
    APs.append(AP)

In [ ]:
plt.figure(figsize=(10, 5), dpi=80)

plt.plot(np.arange(0.5, 1, 0.05), APs, color='black')
plt.axhline(np.mean(APs), color='red', ls='--', label='mAP')
plt.xlabel('IoU')
plt.ylabel('AP')
plt.grid('on')
plt.legend();

print('mAP@[0.5:0.95] = %.2f' % np.mean(APs))
plt.show()

Есть несколько различных определений mAP, которые разняться от соревнования к соревнованию (суть одинаковая, но разница в деталях подхода), поэтому для каждого соревнования лучше использовать их собственные библиотеки для расчета

# DINO — Self-supervised representation learning (with segmentation capabilities)
[Emerging Properties in Self-Supervised Vision Transformers (Caron et al., 2021)](https://arxiv.org/abs/2104.14294)

[Отличное видео объяснение статьи](https://www.youtube.com/watch?v=h3ij3F3cPIk)

[vision_transformer](https://github.com/google-research/vision_transformer)

Для начала подгрузим модель DINO (self-**DI**stillation with **NO** labels)



In [ ]:
# fix random_seed
import torch
import random
import numpy as np
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Compute on cpu or gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!git clone https://github.com/facebookresearch/dino.git

## Сегментация изображений

Теперь загрузим случайную картинку (можно выбрать любую, просто замените ссылку на свою)

In [ ]:
URL = 'https://edunet.kea.su/repo/EduNet-web_dependencies/L12/capybara_image.jpg'
!wget $URL -qO test.jpg

In [ ]:
from PIL import Image

input_img = Image.open('/content/test.jpg')
input_img.resize((400, 300))

Получим результат с помощью DINO

In [ ]:
from IPython.display import clear_output
!python /content/dino/visualize_attention.py --image_path /content/test.jpg 
clear_output()

Посмотрим на картинки, которые генерирует DINO

In [ ]:
from glob import glob
import matplotlib.pyplot as plt

def img_grid(imgs, rows, cols):
    assert len(imgs) == rows * cols
    fig,ax = plt.subplots(nrows=rows, ncols=cols, figsize=(28, 8))
    for num, img in enumerate(imgs):
        img_PIL = Image.open(img)
        ax[num].imshow(img_PIL)
        ax[num].set_xticks([])
        ax[num].set_yticks([])
    plt.subplots_adjust(hspace=0, wspace=0)

img_grid(imgs=sorted(glob('*.png'))[::-1], rows=1, cols=7)
plt.show()

Мы видим 6 карт внимания (self-attention maps - веса слоя self-attention) на 6 головах Visual Transformer. В результате self-supervised обучения по методике DINO, трансформер САМОСТОЯТЕЛЬНО придумал обращать внимание на различные части изображения, таким образом, производя семантическую сегментацию.

Мы можем все эти карты внимания объединить в одно изображение, и просто назначить каждой карте свой цвет, а в качестве прозрачности использовать интенсивность

In [ ]:
import matplotlib.colors as mcolors
from matplotlib import cm

def overlay(img, segmentations):
    img_PIL = Image.open(img)
    fig, ax = plt.subplots(ncols=2, figsize=(10, 10))
    ax[0].imshow(img_PIL)
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    
    ax[1].imshow(img_PIL.convert('LA'), alpha=0.5)
    for num, img in enumerate(segmentations):
        segment_PIL = Image.open(img).convert('LA')
        segment_arr = np.array(segment_PIL)
        colors = [(*cm.tab10(num)[:-1], c) for c in np.linspace(0, 0.75, 100)]
        cmap = mcolors.LinearSegmentedColormap.from_list('mycmap', colors, N=5)
        ax[1].imshow(segment_arr[:, :, 0], cmap=cmap)
    ax[1].set_facecolor('black')
    ax[1].set_xticks([])
    ax[1].set_yticks([])
    plt.subplots_adjust(hspace=0, wspace=0)

overlay('/content/img.png', sorted(glob('*.png'))[:-1])
plt.show()

Видим, что DINO сегментирует разные части нашей картинки на разные семантические группы. В случае с капибарой - это голова, лицо (нос, глаза) и тело.

## Сегментация видео

Вы думали, что на этом все? Нет, DINO может еще удивить. Например, она умеет сегментировать видео

In [ ]:
!sudo pip install --upgrade youtube_dl

Скачаем видео (можно любое)

In [ ]:
URL = 'https://edunet.kea.su/repo/EduNet-web_dependencies/L12/cheetah_video.mp4'
!youtube-dl -o video.mp4 $URL

И сгенерируем сегментированное видео

In [ ]:
from IPython.display import clear_output
!python /content/dino/video_generation.py --input_path /content/video.mp4 --output_path  /content/video_segmented --resize 360 640
!ffmpeg -i /content/video.mp4 -vf scale=640:360 /content/video_scaled.mp4
!ffmpeg \
  -i /content/video_scaled.mp4 \
  -i /content/video_segmented/video.mp4 \
  -filter_complex '[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]' \
  -map '[vid]' \
  -c:v libx264 \
  -crf 23 \
  -preset veryfast \
  output.mp4

clear_output()
print('Complete')

Посмотрим, что получилось. Обратите внимание, эта сеть обучалась в режиме self-supervision!

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Кластеризация

А еще DINO умеет кластеризовать изображения. Выполнять не будем, так как процесс не быстрый, но можем посмотреть на результаты из их статьи

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/clustering_dino.gif" width="400">

# Swin Transformer

[Aug 2021 Swin Transformer: Hierarchical Vision Transformer using Shifted Windows](https://arxiv.org/abs/2103.14030)

Текущий SOTA результат принадлежит модели на базе трансформера.

Применять ViT напрямую для задач сегментации и детектирования - не слишком эффективно, так как при больших размерах patch (16x16) не получится получить точные границы объектов.

А при уменьшении размеров patch будет требоваться все больше ресурсов так как сложность self-attention $O(n^{2})$ пропорциональна квадрату количества элементов на входе.





<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/swin_vs_vit.png" width="600">

Авторы решают проблему при помощи двух усовершенствований.

Self - attention применяется не ко всему изображению сразу, а к его большим фрагментам окнам.

На первый взгляд это возвращает проблему сверток, про которую мы говорили:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/cnn_fail.png" width="600">


Указываем сети куда смотреть, и это помешает оценить взаимное влияние признаков расположенных на разных углах карты.

Чтобы не допустить этой проблемы на каждом следующем transformer -  слое окно сдвигается.








<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/swin_window_shift.png" width="700">



Таким образом, сеть может выучить влияние любого патча на любой. При этом не требуется увеличивать количество входов self-attention блока и количество вычислений не растет. 

Далее, пространственные размеры карт признаков, уменьшаются аналогично тому, как это происходит в сверточных сетях. Для сегментирования и детектирования используется принцип FPN - признака с разных пространственных карт агрегируются для предсказания.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/swin_architecture.png" width="1200">

Patch merging здесь это конкатенация эмбеддингов, с последующей подачей на вход линейного слоя:

Фрагмент из 4-х эмбеддингов 2x2xC конкатенируются. Получаем один тензор 1x1x4C

Затем подаем его на вход линейному слою,  уменьшающему число каналов в 2 раза,
получаем новый эмбеддинг размерностью: 1x1x2C

Таким образом, в отличие от традиционных трансформер архитектур размер embedding здесь меняется.


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L12/swin_result.png" width="500">

Такой подход позволил достичь SOTA результатов как в задаче классификации, так и в задачах детектирования и сегментации. 
Авторы статьи позиционируют Swin - трансформер как backbon решения широкого круга задач CV.

<font size ="6">Список использованной литературы

 COCO

[Подробнее о том, как создать свой COCO датасет с нуля](https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch).

[Подробнее о разметке COCO](https://cocodataset.org/#format-data)

[Что такое  run-length encoding - RLE](https://en.wikipedia.org/wiki/Run-length_encoding)?

[Видео-разбор Run-length encoding](https://www.youtube.com/watch?v=h6s61a_pqfM)

 Семантическая сегментация

[FCN Semantic segmenation](https://towardsdatascience.com/review-fcn-semantic-segmentation-eb8c9b50d2d1)

[Encoder-Decoder with Atrous Separable Convolution for Semantic Image Segmentation (Chen et al., 2018)](https://arxiv.org/abs/1802.02611v3)

[Spatial Pyramid Pooling in Deep Convolutional Networks for Visual Recognition (He et al., 2014)](https://arxiv.org/abs/1406.4729)

 Детектирование

[Recent Progress in Appearance-based Action Recognition (Humphreys et al., 2020)](https://arxiv.org/abs/2011.12619)

[Статья про Selective Search](http://www.huppelen.nl/publications/selectiveSearchDraft.pdf)

[SSD: Single Shot MultiBox Detector (Liu et al., 2015)](https://arxiv.org/abs/1512.02325)

[Focal Loss for Dense Object Detection (Lin et al., 2017)](https://arxiv.org/abs/1708.02002)

[Блог-пост: Что такое Focal Loss и когда его использовать](https://amaarora.github.io/2020/06/29/FocalLoss.html)

[Feature Pyramid Networks for Object Detection (Sergelius et al., 2016)](https://arxiv.org/abs/1612.0314)

[Understanding feature pyramid networks for object detection - FPN](https://jonathan-hui.medium.com/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c)

[You Only Look Once: Unified, Real-Time Object Detection (Redmon et. al., 2015)](https://arxiv.org/abs/1506.02640) 

[YOLO9000: Better, Faster, Stronger (Redmon et. al., 2015)](https://arxiv.org/abs/1612.08242)

[YOLOv3: An Incremental Improvement (Redmon et. al., 2018)](https://arxiv.org/abs/1804.02767)

[YOLOv4: Optimal Speed and Accuracy of Object Detection (Bochkovskiy et al., 2020)](https://arxiv.org/abs/2004.10934)

[YOLOv5 (Glenn Jocher)](https://github.com/ultralytics/yolov5)

[YOLOX: Exceeding YOLO Series in 2021 (Ge et al., 2021)](https://arxiv.org/abs/2107.08430)

 Hard example mining

[Блог пост про Hard Mining Example](https://erogol.com/online-hard-example-mining-pytorch/)

[Training Region-based Object Detectors with Online Hard Example Mining (Shrivastava et al., 2016)](https://arxiv.org/abs/1604.03540)

[Loss Rank Mining: A General Hard ExampleMining Method for Real-time Detectors (Yu et al., 2018)](https://arxiv.org/abs/1804.04606)

 DINO 

[Emerging Properties in Self-Supervised Vision Transformers (Caron et al., 2021)](https://arxiv.org/abs/2104.14294)

[Отличное видео объяснение статьи DINO](https://www.youtube.com/watch?v=h3ij3F3cPIk)

 Другое

[U-Net: Convolutional Networks for Biomedical Image Segmentation (Ronneberger et al., 2015)](https://arxiv.org/abs/1505.04597)

[Блог-пост про 2d свертки с помощью перемножения матриц](https://medium.com/@_init_/an-illustrated-explanation-of-performing-2d-convolutions-using-matrix-multiplications-1e8de8cd2544)